<a href="https://colab.research.google.com/github/umar1196/Image-Data-Analysis/blob/main/CNN_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries**

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms


**Loading the MNIST data and applying transformations**

In [ ]:
# define transformations
transformations = transforms.Compose([transforms.ToTensor(),   # convert image to pytorch tensor
                                      transforms.Normalize((0.5,), (0.5,))]) #normalize the tensor values to have the mean 0.5 and standard deviation 0.5


In [ ]:
# Loading the train data
train_data = datasets.MNIST(root='/data', train=True, download=True, transform=transformations)
train_data

# loading the test data
test_data = datasets.MNIST(root='/data', train=False, download=True, transform=transformations)

100%|██████████| 9912422/9912422 [00:00<00:00, 114484439.87it/s]


Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 28302732.20it/s]


Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 28746981.84it/s]


Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5197961.46it/s]


Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw



In [ ]:
#creating the dataloader for batching and shuffling
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

**Defining the CNN Architecture**

In [ ]:
class CNN(nn.Module):

  def __init__(self, num_classes):
    super (CNN, self).__init__()

    self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3) # first convolutional layer
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d(2,2) # applying max pooling

    self.cnn2 = nn.Conv2d(in_channels=16, out_channels=24, kernel_size=3) # second convolutional layer
    self.relu2 = nn.ReLU()
    self.batch_norm1 = nn.BatchNorm2d(num_features=24) # batch normalization
    self.maxpool2 = nn.MaxPool2d(2,2) # second max pooling

    self.fc1 = nn.Linear(in_features=24*5*5, out_features=128) #fully connetced layer
    self.relu3 = nn.ReLU()
    self.dropout = nn.Dropout2d(0.25)

    self.output_layer = nn.Linear(in_features=128, out_features=10) # output layer


  def forward(self, input_data):
    output = self.cnn1(input_data)
    output = self.relu1(output)
    output = self.maxpool1(output)
    #print(output.shape)

    output = self.cnn2(output)
    output = self.relu2(output)
    output = self.batch_norm1(output)
    output = self.maxpool2(output)
    #print(output.shape)

    output = output.view(-1, 24*5*5)
    output = self.fc1(output)
    output = self.relu3(output)
    output = self.dropout(output)
    output = self.output_layer(output)
    #print(output.shape)

    return output


**Initializing the model, loss function and optimizer**

In [ ]:
model = CNN(10) # model initialization
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # select adama as optimizer
loss_function = nn.CrossEntropyLoss() # loss function cross entropy loss

**Model training** bold text

In [ ]:
epoches = 5

for epoch in range(epoches):
    loss = 0
    acc = 0
    t=0
    for i, (data, label) in enumerate(train_loader):

        optimizer.zero_grad()
        output = model.forward(data)  # forward pass
        loss = loss_function(output, label)

        loss += loss.item()
        _,pred = torch.max(output.data, 1) # obtaining predictions
        acc += torch.sum(pred == label)
        t += label.size(0)

        loss.backward()  # backward pass
        optimizer.step() # updating weights

    print(f"the loss after epoch {epoch} is {loss: .4f}")
    print(f"the accuracy after epoch {epoch} is {acc/t: .4f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


the loss after epoch 0 is  0.1661
the accuracy after epoch 0 is  0.9902
the loss after epoch 1 is  0.0019
the accuracy after epoch 1 is  0.9912
the loss after epoch 2 is  0.0007
the accuracy after epoch 2 is  0.9930
the loss after epoch 3 is  0.0039
the accuracy after epoch 3 is  0.9931
the loss after epoch 4 is  0.0075
the accuracy after epoch 4 is  0.9939


**Model testing**

In [ ]:
with torch.no_grad():
    model.eval()
    acc = 0
    t=0
    for i, (data, label) in enumerate(test_loader):

        output = model(data)
        _,pred = torch.max(output.data, 1)
        acc += torch.sum(pred == label)
        t += label.size(0)

print(f"The accuracy on test data is {acc/t: .4f}")

The accuracy on test data is  0.9924
